# DualVideoMotionTracker

## Paquetes

In [5]:
import cv2
import numpy as np
import pandas as pd

## Funciones

In [6]:
# Initialize lists to store the clicked points
left_points = []
right_points = []

# Colors for the points
colors = [(255, 0, 0), (202, 51, 255), (0, 0, 255), (255, 0, 0), (202, 51, 255), (0, 0, 255)]

# Mouse callback function to store the clicked points and draw a dot
def mouse_callback(event, x, y, flags, param):
    frame_width = param['frame'].shape[1] // 2  # Calculate half of the frame width
    frame_height = param['frame'].shape[0]
    overlay_height = param['overlay'].shape[0]
    etiquetas = ["No detect (Top)", "Feeding (Top)", "No detect (Lateral)", "Feeding (Lateral)"]
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if y < frame_height:
            if len(left_points) < 3 and x < frame_width:
                left_points.append((x, y))
                print(f"Left Point {len(left_points)}: ({x}, {y})")
                # Draw a dot on the frame at the clicked position
                cv2.circle(param['frame'], (x, y), 5, colors[len(left_points) - 1], -1)
            elif len(right_points) < 3 and x >= frame_width:
                right_points.append((x-frame_width, y))
                print(f"Right Point {len(right_points)}: ({x-frame_width}, {y})")
                # Draw a dot on the frame at the clicked position
                cv2.circle(param['frame'], (x, y), 5, colors[len(right_points) + 3 - 1], -1)
        elif frame_height <= y < frame_height + overlay_height:
            # Check if the click is within one of the squares
            square_size = param['overlay'].shape[1] // 4
            square_index = x // square_size
            if x < frame_width:
                if x < frame_width//2:
                    left_points.append((-1, -1))
                    left_points.append((-1, -1))
                    left_points.append((-1, -1))
                else:
                    left_points.append((-2, -2))
                    left_points.append((-2, -2))
                    left_points.append((-2, -2))
            else:
                if x < frame_width + frame_width//2:
                    right_points.append((-1, -1))
                    right_points.append((-1, -1))
                    right_points.append((-1, -1))
                else:
                    right_points.append((-2, -2))
                    right_points.append((-2, -2))
                    right_points.append((-2, -2))
            print(f"Clicked on square {etiquetas[square_index]}")

# Function to process the videos
def process_videos(video_path1, video_path2, output_excel, start_frame1=0, start_frame2=0):
    # Open the video files
    cap1 = cv2.VideoCapture(video_path1)
    cap2 = cv2.VideoCapture(video_path2)

    if not cap1.isOpened() or not cap2.isOpened():
        print("Error: Could not open one or both video files.")
        return

    # Set the starting frames for each video
    cap1.set(cv2.CAP_PROP_POS_FRAMES, start_frame1)
    cap2.set(cv2.CAP_PROP_POS_FRAMES, start_frame2)

    cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
    frac = 0.8
    cv2.resizeWindow("Frame", int(720*frac)*2, int(480*frac + 50))

    all_points = []

    # Create overlay image with 4 squares
    overlay = np.zeros((50, int(720*frac)*2, 3), dtype=np.uint8)
    square_size = overlay.shape[1] // 4
    etiquetas = ["No detect (Top)", "Feeding (Top)", "No detect (Lateral)", "Feeding (Lateral)"]
    for i in range(4):
        top_left = (i * square_size, 0)
        bottom_right = ((i + 1) * square_size, overlay.shape[0])
        cv2.rectangle(overlay, top_left, bottom_right, (255, 255, 255), -1)
        cv2.putText(overlay, f"{etiquetas[i]}", (top_left[0] + 10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    while cap1.isOpened() and cap2.isOpened():
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if not ret1 or not ret2:
            break

        # Resize frames to fit side by side
        frame1 = cv2.resize(frame1, (int(720*frac), int(480*frac)))
        frame2 = cv2.resize(frame2, (int(720*frac), int(480*frac)))

        # Combine frames side by side
        combined_frame = np.hstack((frame1, frame2))

        # Combine the combined frame with the overlay
        display_frame = np.vstack((combined_frame, overlay))

        # Set the mouse callback with the current combined frame and overlay as parameters
        cv2.setMouseCallback("Frame", mouse_callback, {'frame': combined_frame, 'overlay': overlay})

        # Display the frame and wait for user to click six points
        while len(left_points) < 3 or len(right_points) < 3:
            temp_display_frame = display_frame.copy()
            for i, point in enumerate(left_points):
                cv2.circle(temp_display_frame, point, 5, colors[i], -1)
            for i, point in enumerate(right_points):
                cv2.circle(temp_display_frame, (point[0]+int(720*frac), point[1]), 5, colors[i + 3], -1)
            cv2.imshow("Frame", temp_display_frame)

            if cv2.waitKey(1) & 0xFF == 27:  # Exit on ESC key
                cap1.release()
                cap2.release()
                cv2.destroyAllWindows()
                return

        # Store the clicked points for the current frame
        current_points = {
            'Left Point 1': left_points[0] if len(left_points) > 0 else (np.nan, np.nan),
            'Left Point 2': left_points[1] if len(left_points) > 1 else (np.nan, np.nan),
            'Left Point 3': left_points[2] if len(left_points) > 2 else (np.nan, np.nan),
            'Right Point 1': right_points[0] if len(right_points) > 0 else (np.nan, np.nan),
            'Right Point 2': right_points[1] if len(right_points) > 1 else (np.nan, np.nan),
            'Right Point 3': right_points[2] if len(right_points) > 2 else (np.nan, np.nan)
        }
        all_points.append(current_points)

        # Print and store the clicked points for the current frame
        print("Clicked points for the current frame:", left_points + right_points)

        # Clear the points for the next frame
        left_points.clear()
        right_points.clear()

        # Wait before moving to the next frame
        if cv2.waitKey(30) & 0xFF == 27:  # Exit on ESC key
            # Save the points to an Excel file
            df = pd.DataFrame(all_points)
            df.to_excel(output_excel, index=False)
            print(f"Data saved to {output_excel}")
            break

    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

    # Save the points to an Excel file
    df = pd.DataFrame(all_points)
    df.to_excel(output_excel, index=False)
    print(f"Data saved to {output_excel}")

## Prueba

In [ ]:
# Paths to the video files
video_path_dorsal = r"1.mp4"
video_path_lateral = r"2.mp4"

# Output Excel file path
output_excel_path = r"output_points.xlsx"

# Process the videos starting from specific frames
process_videos(video_path_dorsal, video_path_lateral, output_excel_path, start_frame1=6, start_frame2=4)